In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

In [2]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import geemap.foliumap as geemap
import ee
import IPython.display as disp
import folium 
from datetime import date
import geojson
import PIL as image

In [3]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UObgh1RDv0gIT-FxLzH-f7YbvvpwpI21xXiasxr0Xg8&tc=AqB8UfDPz9rQDi9n71rwBOqmJkqsc8EjOYJp-bLzU4g&cc=mKahrI65yZVJN2Q-EuDuBBUu7sf53mxVc4WZfdOfysI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh64kkv58IpzAqipTA8Vwlgt8noYnPdbogks9xS0Ga2mfOKw8z-Jq3k

Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
## Data range
start_date = '2022-01-01'
end_date = '2022-06-30'

In [19]:
Map = geemap.Map()

upper_lake_bhopal = ee.FeatureCollection('users/ferreirawg/upper_lake_bhopal')
geometry = ee.Geometry.Point([77.337352, 23.250177]) 
image = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(geometry) \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20)) \
        .first()\
        .clip(upper_lake_bhopal)
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B8'])

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1= ndwiMasked.toInt()
vectors = ndwiMasked1.reduceToVectors(scale = 30.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)

In [20]:
Map.addLayer(upper_lake_bhopal, {}, 'Upper Lake Bhopal')
Map.addLayer(ndwi, {}, 'NDWI')
Map.addLayer(ndwiMasked1, {'palette': ['00FFFF', '0000FF']}, 'Water')
Map.centerObject(geometry, zoom=12)

In [21]:
#Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
            .filterDate(start_date,end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
            .median()\
            .clip(upper_lake_bhopal)


mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')

ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')


ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')

dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')

In [22]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(mndwi, {'min': -1, 'max': 1, 'palette': ['white', 'black', 'blue']}, 'MNDWI')

In [23]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ph, {'min': 6, 'max': 9, 'palette': ['red', 'green', 'purple']}, 'pH')

Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndti, {'min': -1, 'max': 1, 'palette': ['black', 'yellow', 'red']}, 'NDTI')

Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndci, {'min': -1, 'max': 1, 'palette': ['white', 'black','yellow']}, 'NDCI')


col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate(start_date,end_date) \
.filterBounds(vectors).median()\
.clip(represa_taiacupeba)
temp  = col.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')

Map.addLayer(temp, {'min': -30, 'max': 30, 'palette': ['red', 'white', 'black']}, 'Temperature')

In [24]:
Map.addLayerControl()
Map